In [22]:
import random
from lang import load_data_int_seq
from models.rl.env import IntegerSequenceEnv
from models.rl.agents.agent_mcts.main import train_mcts


%load_ext autoreload
%autoreload 2

MAX_PENALTY_MAGNITUDE = 999.0

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [23]:
output_lang, input_lang, train, X_test, y_test = load_data_int_seq()
train = train[:1]

In [24]:
env = IntegerSequenceEnv({"data": train, "output_length": 9, "input_lang": input_lang, "output_lang": output_lang})

self.state
[-1, -1, -1, -1, -1, -1, -1, -1, -1, 2, 3, 4, 5, 6, 7, 8, 9, 10]


In [25]:
env.action_space.n

14

In [26]:
env.observation_space.shape[0]

18

In [27]:
train_mcts(env, num_epochs=2)

Testing Agent

Training Episodes: 0
[-1, -1, -1, -1, -1, -1, -1, -1, -1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
Step: 0
Return: 0

Training Episodes: 0
[11, -1, -1, -1, -1, -1, -1, -1, -1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
Step: 1
Return: -999.0
[array([-1, -1, -1, -1, -1, -1, -1, -1, -1,  2,  3,  4,  5,  6,  7,  8,  9,
       10]), array([ 7, -1, -1, -1, -1, -1, -1, -1, -1,  2,  3,  4,  5,  6,  7,  8,  9,
       10], dtype=int64), array([ 7,  6, -1, -1, -1, -1, -1, -1, -1,  2,  3,  4,  5,  6,  7,  8,  9,
       10], dtype=int64), array([ 7,  6,  2, -1, -1, -1, -1, -1, -1,  2,  3,  4,  5,  6,  7,  8,  9,
       10], dtype=int64)]
[array([0.03030303, 0.12121212, 0.06060606, 0.12121212, 0.12121212,
       0.06060606, 0.06060606, 0.06060606, 0.06060606, 0.06060606,
       0.06060606, 0.06060606, 0.06060606, 0.06060606], dtype=float32), array([0.02380952, 0.16666667, 0.02380952, 0.2857143 , 0.2857143 ,
       0.02380952, 0.02380952, 0.02380952, 0.02380952, 0.02380952,
       0.02380952, 0.02380952, 0.02380

ValueError: shape mismatch: value array of shape (4,18) could not be broadcast to indexing result of shape (4,)

# HillClimbingEnv

In [ ]:
from models.rl.agents.agent_mcts.env_archive.hill_climbing_env import HillClimbingEnv

In [ ]:
env = HillClimbingEnv()

In [ ]:
train_mcts(env, num_epochs=2)